# Pivot and Validate Precinct Level Election Results

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

### Import the datasets

In [2]:
statewide_election_results_file_path = '20181106__ny__general__precinct.csv'
statewide_results_df = pd.read_csv(statewide_election_results_file_path,low_memory=False)
statewide_results_df.head(2)

,county,precinct,office,district,candidate,party,votes,election_day,absentee,machine_votes,absentee_hc,affidavit
0,Chemung,Elmira 0901,Governor,NaN,Andrew Cuomo,Dem,139,NaN,NaN,NaN,NaN,NaN
1,Chemung,Elmira 0902,Governor,NaN,Andrew Cuomo,Dem,104,NaN,NaN,NaN,NaN,NaN


In [3]:
official_county_level_election_results_file_path = 'official_county_level_results_2018Governor.csv'
official_county_level_election_results_df = pd.read_csv(official_county_level_election_results_file_path,low_memory=False)
official_county_level_election_results_df = official_county_level_election_results_df.set_index('County')
official_county_level_election_results_df.head(2)

,Andrew M. Cuomo and Kathy C. Hochul (DEM),Marc Molinaro and Julie Killian (REP),Marc Molinaro and Julie Killian (CON),Howie Hawkins and Jia Lee (GRE),Andrew M. Cuomo and Kathy C. Hochul (WOR),Andrew M. Cuomo and Kathy C. Hochul (IND),Andrew M. Cuomo and Kathy C. Hochul (WEP),Marc Molinaro and Julie Killian (REF),Larry Sharpe and Andrew C. Hollister (LBT),Stephanie A. Miner and Michael J. Volpe (SAM),Blank,Void,Scattering,Total Votes by County
County,,,,,,,,,,,,,,
Albany County,54909,38357,6725,3543,2489,1408,886,835,1867,3007,2807,268,195,"117,296"
Allegany County,3254,9234,833,183,80,106,46,65,963,117,462,3,7,"15,353"


The next cell aliases `statewide_results_df` as `df` and `official_county_level_election_results_df` as `odf` because typing fewer characters allows for faster data exploration.  

In [4]:
df = statewide_results_df.copy()
odf = official_county_level_election_results_df.copy()

### Clean `votes`

In [5]:
def remove_commas(df, col_lst):
    """
    remove commas from the string representation of numbers in the cols in col_lst
    :df: DataFrame object
    :col_lst: list of strings that are each the name of a column :df:
    """
    for col in col_lst:
        if df.dtypes[col] == "object":
            df[col] = df[col].map(lambda s: s.replace(",", ""))
        df = df.astype({col: "float"})
        df = df.astype({col: "int64"})
    return df

df = remove_commas(df[df.votes.notna()], ['votes'])

/Users/baxterdemers/ny2018/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### Clean `county`

In [6]:
df.county = df.county.str.strip() + ' County'

### Clean `office`

In [7]:
def clean_office(office):
    if 'gov' in office.lower():
        return 'Governor'
    if office == 'U.S.Senate':
        return 'U.S. Senate'
    if 'U.S.House' in office:
        return 'U.S. House'
    return office.title().strip()

df.office.map(clean_office).unique()

array(['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate',
       'U.S. House', 'State Senate', 'State Assembly'], dtype=object)

In [8]:
df.office = df.office.map(clean_office)

In [9]:
offices = list(df.office.unique())
print(len(offices))
print(offices)

7
['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate', 'U.S. House', 'State Senate', 'State Assembly']


Ensure that all counties data for each office

In [10]:
county_to_offices = df.groupby('county').office.unique()
county_to_missing_offices = county_to_offices.apply(lambda x: set(offices).difference(x))
county_to_missing_offices[county_to_missing_offices != set()]

county
Albany County                                {State Assembly}
Allegany County                                    {Governor}
Broome County                                {State Assembly}
Chautauqua County                            {State Assembly}
Chemung County                               {State Assembly}
Clinton County                               {State Assembly}
Columbia County               {Comptroller, Attorney General}
Cortland County                              {State Assembly}
Erie County                                  {State Assembly}
Genesee County                                     {Governor}
Kings County                                 {State Assembly}
Madison County                               {State Assembly}
Montgomery County                            {State Assembly}
New York County                              {State Assembly}
Niagara County                                  {U.S. Senate}
Oneida County                                {State Assembly}
O

In [11]:
federal_offices = ['Governor', 'U.S. Senate', 'U.S. House']
county_to_missing_federal_offices = county_to_offices.apply(lambda x: set(federal_offices).difference(x))
counties_missing_federal_offices = county_to_missing_federal_offices[county_to_missing_federal_offices != set()]
print(len(counties_missing_federal_offices))
counties_missing_federal_offices

7


county
Allegany County                {Governor}
Genesee County                 {Governor}
Niagara County              {U.S. Senate}
Orange County      {U.S. House, Governor}
Saratoga County              {U.S. House}
Schuyler County                {Governor}
Suffolk County                 {Governor}
Name: office, dtype: object

Seems like Open Elections has more up to date county level files here: https://github.com/openelections/openelections-data-ny/tree/master/2018/counties

* Allegany County                {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__allegany__precinct.csv
* Genesee County                 {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__genesee__precinct.csv
* Niagara County              {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__niagara__precinct.csv
* Orange County      {U.S. House, X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__orange__precinct.csv
* Saratoga County              {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__saratoga__precinct.csv
* Schuyler County                {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__schuyler__precinct.csv
* Suffolk County                 {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__suffolk__precinct.csv 

All set with the expection of missing results for the U.S. House race in Orange County.

### Getting results for U.S. House in Orange County

1. Downloaded `data/parse_results/GE18 -CD 18th.pdf` from https://www.orangecountygov.com/DocumentCenter/View/11663/GE18--CD-18th
2. Used https://pdftables.com/ to convert the pdf to a csv (`data/parse_results/GE18 -CD 18th.csv`)
3. Made some manual tweaks to clean up the handwork of the pdf to csv converter in Numbers (like Excel for Mac OS). Saved it locally at `data/parse_results/GE18-CD-18th-cleaned.csv`
4. Imported below

Since this data is already pivoted, I will incoperate it after I pivot the rest of the data.

In [53]:
df_Orange_HOR = pd.read_csv('data/parse_results/GE18-CD-18th-cleaned.csv')
df_Orange_HOR

,precinct,DEM,REP,CON,WOR,IND,WEP,REF,WRITE IN,BLANK
0,Middletown - Ward 1-1,159,67,7,6,4,2,1,0,11
1,Middletown - Ward 1-2,79,34,3,1,0,0,0,0,3
2,Middletown - Ward 1-3,158,47,7,3,2,1,2,0,15
3,Middletown - Ward 1-4,36,19,0,2,1,0,0,0,3
4,Middletown - Ward 1-5,139,53,7,6,1,0,1,0,9
...,...,...,...,...,...,...,...,...,...,...
329,Woodbury 6,202,191,27,7,6,3,4,0,34
330,Woodbury 7,192,117,13,3,9,2,8,0,15
331,Woodbury 8,175,211,27,3,5,1,2,0,13
332,Woodbury 9,129,36,7,0,1,1,1,0,10


### Patch in Counties that are missing election results

For whatever reason, Open Elections has more complete results at the county level. Counties missing election results for federal offices: 
* Allegany County                {Governor}
* Genesee County                 {Governor}
* Niagara County              {U.S. Senate}
* Orange County      {Governor}
* Saratoga County              {U.S. House}
* Schuyler County                {Governor}
* Suffolk County                 {Governor}

In [13]:
df_Allegany = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__allegany__precinct.csv')
df_Genesee = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__genesee__precinct.csv')
df_Niagara = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__niagara__precinct.csv')
df_Orange = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__orange__precinct.csv')
df_Saratoga = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__saratoga__precinct.csv')
df_Schuyler = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__schuyler__precinct.csv')
df_Suffolk = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__suffolk__precinct.csv')

In [14]:
county_lst = ['Allegany', 'Genesee', 'Niagara', 'Orange', 'Saratoga', 'Schuyler', 'Suffolk']
county_df_lst = [df_Allegany, df_Genesee, df_Niagara, df_Orange, df_Saratoga, df_Schuyler,df_Suffolk]

In [15]:
df = df[~df.county.isin(county_lst)]
df = df.append(county_df_lst)

In [16]:
df[df.county == 'Allegany'].office.unique()

array(['State Assembly', 'State Senate', 'U.S. House', 'U.S. Senate',
       'GOVERNOR/ LT.', 'COMPTROLLER', 'ATTORNEY GENERAL'], dtype=object)

### Repeat cleaning

In [17]:
def clean_office(office):
    if 'gov' in office.lower():
        return 'Governor'
    if office == 'U.S.Senate':
        return 'U.S. Senate'
    if 'U.S.House' in office:
        return 'U.S. House'
    return office.title().strip()

df.office = df.office.map(clean_office)
df = df[df.office.isin(offices)]
df.office.unique()

array(['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate',
       'U.S. House', 'State Senate', 'State Assembly'], dtype=object)

In [18]:
df.loc[df.county.isin(county_lst), 'county'] = df.loc[df.county.isin(county_lst),'county'].str.strip() + ' County'

Ensure that all counties data for each office

In [19]:
county_to_offices = df.groupby('county').office.unique()
county_to_missing_offices = county_to_offices.apply(lambda x: set(offices).difference(x))
county_to_missing_offices = county_to_missing_offices[county_to_missing_offices != set()]
print(len(county_to_missing_offices))
county_to_missing_offices

21


county
Albany County                         {State Assembly}
Broome County                         {State Assembly}
Chautauqua County                     {State Assembly}
Chemung County                        {State Assembly}
Clinton County                        {State Assembly}
Columbia County        {Comptroller, Attorney General}
Cortland County                       {State Assembly}
Erie County                           {State Assembly}
Kings County                          {State Assembly}
Madison County                        {State Assembly}
Montgomery County                     {State Assembly}
New York County                       {State Assembly}
Oneida County                         {State Assembly}
Orange County             {State Assembly, U.S. House}
Queens County                            {Comptroller}
Richmond County                       {State Assembly}
St. Lawrence County                   {State Assembly}
Tompkins County                       {State Assembly}
War

In [20]:
federal_offices = ['Governor', 'U.S. Senate', 'U.S. House']
county_to_missing_federal_offices = county_to_offices.apply(lambda x: set(federal_offices).difference(x))
counties_missing_federal_offices = county_to_missing_federal_offices[county_to_missing_federal_offices != set()]
print(len(counties_missing_federal_offices))
counties_missing_federal_offices

1


county
Orange County    {U.S. House}
Name: office, dtype: object

Exclude non-federal offices because so much source data is missing

In [21]:
df = df[df.office.isin(federal_offices)]

### Clean `party`

The oficial parties in New York are [defined as](https://www.elections.ny.gov/CFCommittees.html)

- Democratic
- Republican
- Conservative
- Working Families
- Green
- Libertarian
- Independence
- SAM

Since the official results also included `WEP (Women's Equality Party)` and `REF (Reform Party of New York State)` I will include those as well.

In [23]:
df.party.nunique()

75

In [24]:
official_party_to_abbreviation = {
    "Democratic":"DEM",
    "Republican":"REP",
    "Conservative":"CON",
    "Working Families":"WOR",
    "Green":"GRE",
    "Libertarian":"LBT",
    "Independence":"IND",
    "SAM": "SAM",
    "Women's Equality Party":"WEP",
    "Reform Party":"REF",
}

parties = set(official_party_to_abbreviation.values())
misc = {'BLANK, VOID OR SCATTERED', 'WRITEIN', 'OVERVOTE', 'UNDERVOTE', 'UNQUALIFIED WRITE-INS'}

In [25]:
party_lookup_table = {
    'DEMOCRATIC':'DEM',
    '0':'BLANK, VOID OR SCATTERED',
    '1635':'BLANK, VOID OR SCATTERED',
    '4328':'BLANK, VOID OR SCATTERED',
    'BLANK & VOID':'BLANK, VOID OR SCATTERED',
    'BLANKS':'BLANK, VOID OR SCATTERED',
    'BLANKS & VOID':'BLANK, VOID OR SCATTERED',
    'IND': 'IND',
    'REP': 'REP',
 'Write': 'Write',
 'DEM': 'DEM',
 'CON': 'CON',
 'SAM': 'SAM',
     'GRE': 'GRE',
 'WOR': 'WOR',
 'REF': 'REF',
 'WEP': 'WEP',
 'LBT': 'LBT',
 'WRI':'WRITEIN',
 'WRITE IN':'WRITEIN',
 'WRITE INS':'WRITEIN',
 'WRITE-IN':'WRITEIN',
 'WRITE-INS':'WRITEIN',
 'WRITEIN':'WRITEIN',
 'WRITEINS':'WRITEIN',
     'VOID':'BLANK, VOID OR SCATTERED',
 'VOIDS':'BLANK, VOID OR SCATTERED',
 'VOIDS/BLANKS':'BLANK, VOID OR SCATTERED',
    'GREEN':'GRE',
 'GRN':'GRE',
     'BLANK':'BLANK, VOID OR SCATTERED',
 'BLANK VOTES':'BLANK, VOID OR SCATTERED',
 'BLANK/VOID':'BLANK, VOID OR SCATTERED',
     'LIB':'LBT',
 'LIBERTARIAN':'LBT',
    'SCATTER':'BLANK, VOID OR SCATTERED',
 'SCATTERED':'BLANK, VOID OR SCATTERED',
 'SCATTERING':'BLANK, VOID OR SCATTERED',
 'SCATTERINGS':'BLANK, VOID OR SCATTERED',
'CONSERVATIVE':'CON',
         'OVER':'OVERVOTE',
 'OVER VOTES':'OVERVOTE',
 'OVERVOTE':'OVERVOTE',
 'OVERVOTES':'OVERVOTE',
 'UNDER':'UNDERVOTE',
 'UNDER VOTES':'UNDERVOTE',
 'UNDER/OVER':'UNDERVOTE',
 'UNDER/OVER VOTES':'UNDERVOTE',
 'UNDERVOTE':'UNDERVOTE',
 'UNDERVOTES':'UNDERVOTE',
    'REPUBLICAN':'REP',
    'REFORM':'REF',
   'REFORM PARTY':'REF',
    'INDEPENDENCE':'IND',
    'WFP':'WOR',
    'WF':'WOR',
    "WOMEN'S EQUALITY PARTY":'WEP',
    "WOMEN'S EQUALITY":"WEP",
    # ballotpedia lookups
    'GLOGOWSKI':'LBT',
    "CHRISTOPHER B. GARVEY":'LBT',
    'SEAMANS':'DEM',
    'DINAPOLI':'DEM',
    'GILLIBRAND':'DEM',
    'JAMES': 'DEM',
    'COLLINS':'REP',
    'TRICHTER':'REP',
    'CC FARLEY':'REP',
    'WOFFORD':'REP',
    'RANZENHOFER':'REP',
    'HAWLEY':'REP',
    'MCMURRAY':'DEM',
}

def clean_party(party):
    if (type(party) != str):
        return 'nS'
    cleaned_party = party.strip().upper()
    looked_up_party = party_lookup_table.get(cleaned_party, cleaned_party)
    return official_party_to_abbreviation.get(looked_up_party.title(), looked_up_party)

In [26]:
df = df.loc[~(df.party.str.upper().isin([
        'BALLOTS CAST',
        'TOTAL',
        'TOTAL BALLOTS',
        'TOTAL VOTES',
        'REGISTERED VOTERS',
        'FEDERAL',
        'FEDERAL BALLOTS',
        'FEDERAL VOTES',
        'STATE BALLOTS',
        'STATE VOTES',
        'REGISTERED',
        'PUBLIC COUNTER',]))
        ]

In [27]:
df['party'] = df.party.map(clean_party)
df = df[df.party.isin(parties)]
value_counts = df.groupby('party').votes.sum()
print(value_counts[value_counts > 1000].shape)
value_counts[value_counts > 1000].sort_values().tail(60)

(10,)


party
SAM       62688
LBT      104572
REF      109624
WEP      132514
GRE      139722
IND      241758
WOR      565943
CON      844918
REP     6232204
DEM    13150530
Name: votes, dtype: int64

In [28]:
df.office.unique()

array(['Governor', 'U.S. Senate', 'U.S. House'], dtype=object)

In [29]:
office_to_abbreviation = {
    'Governor':'Gov', 
    'Comptroller':'Compt', 
    'Attorney General':'AG',
    'U.S. Senate':'Sen',
    'U.S. House':'HOR',
    'State Senate':'StSen', 
    'State Assembly':'StAssy'
}

In [30]:
df.office = df.office.map(office_to_abbreviation)

In [31]:
df.head()

,county,precinct,office,district,candidate,party,votes,election_day,absentee,machine_votes,absentee_hc,affidavit,Unnamed: 7,Unnamed: 8
0,Chemung County,Elmira 0901,Gov,NaN,Andrew Cuomo,DEM,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chemung County,Elmira 0902,Gov,NaN,Andrew Cuomo,DEM,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chemung County,Elmira 0903,Gov,NaN,Andrew Cuomo,DEM,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chemung County,Elmira 0904,Gov,NaN,Andrew Cuomo,DEM,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chemung County,Elmira 0905,Gov,NaN,Andrew Cuomo,DEM,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
piv = pd.pivot_table(df, index = ['county', 'precinct'], columns = ['office','party'], values = ['votes'], aggfunc = np.sum)

In [33]:
piv.columns = [office + '_' + party for _,office,party in piv.columns]
piv.head()

Gov_CON  Gov_DEM  Gov_GRE  Gov_IND  Gov_LBT  \
county        precinct                                                          
Albany County 0001 ALBANY W1 ED1      1.0     35.0      3.0      0.0      1.0   
              0002 ALBANY W1 ED2     11.0    148.0     15.0      1.0      3.0   
              0003 ALBANY W1 ED3     11.0    212.0      4.0      1.0      5.0   
              0004 ALBANY W1 ED4      5.0    220.0      4.0      4.0      4.0   
              0005 ALBANY W1 ED5      0.0      2.0      1.0      0.0      0.0   

                                  Gov_REF  Gov_REP  Gov_SAM  Gov_WEP  Gov_WOR  \
county        precinct                                                          
Albany County 0001 ALBANY W1 ED1      0.0      9.0      2.0      0.0      3.0   
              0002 ALBANY W1 ED2      2.0     32.0      9.0      1.0     11.0   
              0003 ALBANY W1 ED3      0.0     55.0      5.0      3.0     10.0   
              0004 ALBANY W1 ED4      3.0     30.0      7.0      1.0     12.0   
              0005 ALBANY W1 ED5      0.0      0.0      0.0      0.0      0.0   

                                  ...  HOR_REP  HOR_WEP  HOR_WOR  Sen_CON  \
county        precinct            ...                                       
Albany County 0001 ALBANY W1 ED1  ...      7.0      1.0      5.0      1.0   
              0002 ALBANY W1 ED2  ...     20.0      6.0     21.0      5.0   
              0003 ALBANY W1 ED3  ...     41.0      1.0     11.0     11.0   
              0004 ALBANY W1 ED4  ...     18.0      5.0     18.0      6.0   
              0005 ALBANY W1 ED5  ...      0.0      0.0      0.0      0.0   

                                  Sen_DEM  Sen_IND  Sen_REF  Sen_REP  Sen_WEP  \
county        precinct                                                          
Albany County 0001 ALBANY W1 ED1     40.0      0.0      0.0      8.0      0.0   
              0002 ALBANY W1 ED2    176.0      4.0      1.0     23.0      4.0   
              0003 ALBANY W1 ED3    228.0      5.0      0.0     40.0      5.0   
              0004 ALBANY W1 ED4    241.0      5.0      3.0     15.0      4.0   
              0005 ALBANY W1 ED5      4.0      0.0      0.0      0.0      0.0   

                                  Sen_WOR  
county        precinct                     
Albany County 0001 ALBANY W1 ED1      6.0  
              0002 ALBANY W1 ED2     22.0  
              0003 ALBANY W1 ED3     10.0  
              0004 ALBANY W1 ED4     15.0  
              0005 ALBANY W1 ED5      0.0  

[5 rows x 25 columns]

In [34]:
piv.groupby('county').sum()

,Gov_CON,Gov_DEM,Gov_GRE,Gov_IND,Gov_LBT,Gov_REF,Gov_REP,Gov_SAM,Gov_WEP,Gov_WOR,...,HOR_REP,HOR_WEP,HOR_WOR,Sen_CON,Sen_DEM,Sen_IND,Sen_REF,Sen_REP,Sen_WEP,Sen_WOR
county,,,,,,,,,,,,,,,,,,,,,
Albany County,6725.0,54909.0,3543.0,1408.0,1867.0,835.0,38357.0,3007.0,886.0,2489.0,...,32326.0,1640.0,4781.0,6034.0,70182.0,2655.0,597.0,29825.0,1395.0,3883.0
Allegany County,833.0,12488.0,183.0,106.0,963.0,65.0,0.0,117.0,46.0,80.0,...,2252.0,0.0,0.0,1784.0,9688.0,444.0,128.0,16610.0,198.0,590.0
Bronx County,2407.0,253712.0,2828.0,2143.0,1074.0,271.0,20401.0,593.0,503.0,4198.0,...,14712.0,1204.0,8339.0,2590.0,249485.0,2369.0,331.0,20112.0,698.0,5171.0
Broome County,3244.0,29065.0,1320.0,931.0,2000.0,474.0,32865.0,802.0,379.0,1144.0,...,29452.0,469.0,1774.0,2910.0,37053.0,1588.0,331.0,27656.0,597.0,1907.0
Cattaraugus County,1629.0,6545.0,307.0,265.0,1060.0,159.0,13240.0,192.0,78.0,146.0,...,13173.0,239.0,444.0,1556.0,9199.0,556.0,114.0,11586.0,179.0,497.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Washington County,1395.0,5964.0,453.0,198.0,571.0,154.0,11330.0,199.0,82.0,167.0,...,10263.0,182.0,437.0,1255.0,9248.0,499.0,89.0,8596.0,168.0,550.0
Wayne County,2765.0,8655.0,435.0,255.0,1648.0,262.0,16547.0,461.0,114.0,217.0,...,15918.0,338.0,625.0,2712.0,12374.0,0.0,218.0,14592.0,267.0,660.0
Westchester County,10295.0,212833.0,3738.0,4692.0,1892.0,1100.0,90785.0,2576.0,1865.0,3295.0,...,10079.0,3753.0,9666.0,10789.0,221750.0,4190.0,967.0,85222.0,2277.0,5956.0


### Add in HOR results for Orange County

Need to add a multi index to `df_Orange_HOR` to allow for indexed assignment with `piv` (which has a multi index) 

In [55]:
df_Orange_HOR = df_Orange_HOR.set_index('precinct')
df_Orange_HOR['county'] = 'Orange County'
df_Orange_HOR.set_index('county', append=True, inplace=True)
df_Orange_HOR = df_Orange_HOR.reorder_levels(['county', 'precinct'])
df_Orange_HOR.head()

,,DEM,REP,CON,WOR,IND,WEP,REF,WRITE IN,BLANK
precinct,county,,,,,,,,,
Middletown - Ward 1-1,Orange County,159,67,7,6,4,2,1,0,11
Middletown - Ward 1-2,Orange County,79,34,3,1,0,0,0,0,3
Middletown - Ward 1-3,Orange County,158,47,7,3,2,1,2,0,15
Middletown - Ward 1-4,Orange County,36,19,0,2,1,0,0,0,3
Middletown - Ward 1-5,Orange County,139,53,7,6,1,0,1,0,9


Along the same lines, sorting the indices is needed so they line up.

In [66]:
piv = piv.sort_index()
df_Orange_HOR = df_Orange_HOR.sort_index()
list(piv.loc['Orange County'].index) == list(df_Orange_HOR.loc['Orange County','DEM'].index)

True

Now I can make the substitutions

In [69]:
[x for x in piv.columns if 'HOR' in x]

['HOR_CON',
 'HOR_DEM',
 'HOR_GRE',
 'HOR_IND',
 'HOR_REF',
 'HOR_REP',
 'HOR_WEP',
 'HOR_WOR']

In [71]:
piv.loc['Orange County', 'HOR_CON'] = df_Orange_HOR['CON']
piv.loc['Orange County', 'HOR_DEM'] = df_Orange_HOR['DEM']
piv.loc['Orange County', 'HOR_GRE'] = 0
piv.loc['Orange County', 'HOR_IND'] = df_Orange_HOR['IND']
piv.loc['Orange County', 'HOR_REF'] = df_Orange_HOR['REF']
piv.loc['Orange County', 'HOR_REP'] = df_Orange_HOR['REP']
piv.loc['Orange County', 'HOR_WEP'] = df_Orange_HOR['WEP']
piv.loc['Orange County', 'HOR_WOR'] = df_Orange_HOR['WOR']

In [73]:
piv.loc['Orange County'].head(2)

,Gov_CON,Gov_DEM,Gov_GRE,Gov_IND,Gov_LBT,Gov_REF,Gov_REP,Gov_SAM,Gov_WEP,Gov_WOR,...,HOR_REP,HOR_WEP,HOR_WOR,Sen_CON,Sen_DEM,Sen_IND,Sen_REF,Sen_REP,Sen_WEP,Sen_WOR
precinct,,,,,,,,,,,,,,,,,,,,,
Blooming Grove 1-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,64.0,0.0,1.0,22.0,232.0,2.0,6.0,136.0,0.0,4.0
Blooming Grove 1-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,183.0,0.0,8.0,40.0,520.0,20.0,12.0,350.0,10.0,20.0


## TODO
* [_] Validate Results for Gov
* [_] Pull more county level official results and do sen + HOR?

## Validation

Compare the precinct level election results to official county level election results released by New York's Secretary of State

### Clean up the pivoted data

In [ ]:
piv = remove_commas(piv.fillna(0), piv.columns)

In [ ]:
res_gov = piv.groupby('county').sum()[[x for x in piv.columns if x.startswith('Gov')]]
res_gov.head()

In [ ]:
res_gov.columns = [x.split('_')[1] for x in res_gov.columns]
res_gov.head()

In [ ]:
o_gov = odf[[x for x in odf.columns if '(' in x]]
o_gov.columns = [x.split('(')[1][:3] for x in o_gov.columns]
o_gov.head()

In [ ]:
party_lst = list(parties)
party_lst.sort()
party_lst

Order columns the same way in both data sets

In [ ]:
o_gov = o_gov[party_lst]
o_gov.head(2)

In [ ]:
res_gov = res_gov[party_lst]
res_gov.head(2)

Allegany seemingly missing results for the govenors race. Will now check out the `df` for other `county, office` tuples that are missing data.

In [ ]:
cov = df.groupby(['county', 'office']).votes.sum().sort_values()

In [ ]:
cov['Allegany']

In [ ]:
cov

In [ ]:
df.groupby('county').office.unique()